In [ ]:
import umap
from sklearn.manifold import TSNE

import sys
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import loompy
import velocyto as vcy
import logging
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from statsmodels.nonparametric.smoothers_lowess import lowess
from scipy.interpolate import interp1d


In [ ]:
output_dir = "."

In [ ]:
loomInput = f"{output_dir}/velocyto_input.loom"
vlm = vcy.VelocytoLoom(loomInput)

In [ ]:
diff_map = pd.DataFrame(vlm.ca['X_diffmap'])
vlm.ca['X_diffmap']  diff_map.iloc[:,[1,2]].to_numpy()
vlm.dc =  vlm.ca['X_diffmap']
vlm.ts =  vlm.ca['X_diffmap']

In [ ]:
clusters = np.array([str(i) for i in vlm.ca["Clusters"]])
colors20 = ["#E41A1C", "#377EB8", "#4DAF4A", "#984EA3", "#FF7F00", "#A65628", "#F781BF", "#999999", "#1B9E77", "#D95F02", "#7570B3", "#E7298A", "#66A61E", "#E6AB02", "#A6761D"]
vlm.set_clusters(clusters, cluster_colors_dict={val:colors20[idx] for idx, val in enumerate(np.unique(clusters))})

vlm.score_detection_levels(min_expr_counts=40, min_cells_express=30)
vlm.filter_genes(by_detection_levels=True)

vlm.score_cv_vs_mean(2000, plot=True, max_expr_avg=35)
vlm.filter_genes(by_cv_vs_mean=True)

vlm.score_detection_levels(min_expr_counts=0, min_cells_express=0, min_expr_counts_U=25, min_cells_express_U=20)
vlm.score_cluster_expression(min_avg_U=0.01, min_avg_S=0.08)
vlm.filter_genes(by_detection_levels=True, by_cluster_expression=True)

vlm._normalize_S(relative_size=vlm.initial_cell_size, target_size=np.mean(vlm.initial_cell_size))
vlm._normalize_U(relative_size=vlm.initial_Ucell_size, target_size=np.mean(vlm.initial_Ucell_size))

vlm.perform_PCA()
n_comps = np.where(np.diff(np.diff(np.cumsum(vlm.pca.explained_variance_ratio_))>0.002))[0][0]

In [ ]:
k = 500
vlm.knn_imputation(n_pca_dims=n_comps, k=k, balanced=True, b_sight=np.minimum(k*8, vlm.S.shape[1]-1), b_maxl=np.minimum(k*4, vlm.S.shape[1]-1), n_jobs=16)
vlm.fit_gammas(limit_gamma=False, fit_offset=False)
vlm.predict_U()
vlm.calculate_velocity()
vlm.calculate_shift(assumption="constant_velocity")
vlm.extrapolate_cell_at_t(delta_t=1.)

In [ ]:
vlm.ra["Gene"] = vlm.ra["var_names"]
clusters = np.array([str(i) for i in vlm.ca["Clusters"]])
vlm.set_clusters(clusters, cluster_colors_dict={val:colors20[idx] for idx, val in enumerate(np.unique(clusters))})

In [ ]:
vlm.estimate_transition_prob(hidim="Sx_sz", embed="ts", transform="sqrt", psc=1, n_neighbors=2000, knn_random=True, sampled_fraction=0.5, n_jobs=16)
vlm.calculate_embedding_shift(sigma_corr = 0.05, expression_scaling=False)
vlm.calculate_grid_arrows(smooth=0.8, steps=(40, 40), n_neighbors=500)

In [ ]:
kwarg_plot = {"alpha": 1.0, "s": 50, "edgecolor": "0.8", "lw": 0.0}
plt.figure(None,(10, 10))
vcy.scatter_viz(vlm.flow[:,0], vlm.flow[:,1], c="k", **kwarg_plot)
vcy.scatter_viz(vlm.embedding[:,0], vlm.embedding[:,1], c=vlm.colorandum, **kwarg_plot)
plt.axis("off")
plt.savefig(f"{output_dir}/velocyto_diffmap_scatter.png")
plt.close()

In [ ]:
plt.figure(None,(10, 10))
vlm.plot_grid_arrows(quiver_scale=1, scatter_kwargs_dict={"alpha":0.5, "lw":0.35, "edgecolor":"0.4", "s":15, "rasterized":True}, min_mass=10, angles='xy', scale_units='xy', headaxislength=2.75, headlength=5, headwidth=4.8, minlength=0.5, scale_type = "absolute") #absolute relative
plt.savefig(f"{output_dir}/velocyto.png")
plt.close()